Install necessary python modules

In [ ]:
!pip install pyjugex
!pip install nibabel
!pip install ebrains-atlascore

Import the modules

In [ ]:
from pyjugex import PyjugexAnalysis
import nibabel as nib
import pyjugex
import ebrains_atlascore
import numpy

Set up function to merge regions of interest

In [5]:
def threshold_function(image_data, threshold):
    image_data[numpy.where(image_data < float(threshold))] = 0
    return image_data

def merge_list(areaList, threshold):
    combined_mask = 0
    
    for area in areaList:
        image_mask = threshold_function(area.data, threshold) != 0

        combined_mask = numpy.logical_or(combined_mask, image_mask)
    
    return combined_mask

Parameters selected by you:

In [4]:
Area1 = $AREA1$
Area2 = $AREA2$
genelist = $GENELIST$
threshold = $THRESHOLD$
n_rep = $REPS$

SyntaxError: invalid syntax (<ipython-input-4-f682477b2401>, line 1)

Set up individual thresholds if wished:

Retrieve regions for Area 1:

In [ ]:
area1_rois = []

for region in Area1:
    area2_rois.append(ebrains-atlascore.regions.get_region_by_name('colin', 'JuBrain Cytoarchitectonic Atlas', region))

Retrieve regions for Area 2:

In [ ]:
area2_rois = []

for region in Area2:
    area2_rois.append(ebrains-atlascore.regions.get_region_by_name('colin', 'JuBrain Cytoarchitectonic Atlas', region))

Merge regions for area 1 and area 2

In [ ]:
area1_mask = merge_list(area1_rois)
area2_mask = merge_list(area2_rois)

In [ ]:
gene_list=$$GENELIST$$
n_rep = 1000

In [ ]:
analysis = PyjugexAnalysis(
  n_rep=1000,
  gene_list=gene_list,
  roi1 = hoc1_nii,
  roi2 = hoc2_nii
)

In [ ]:
analysis.differential_analysis() # Go grab a coffee

In [ ]:
print(analysis.anova.result)